In [1]:

import pandas as pd
import os, random, sys, copy
import torch, torch.nn as nn, numpy as np
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

import csv
import os
import re
import neattext.functions as nfx

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

import seaborn as sns
from sklearn.pipeline import Pipeline

DataSets Used:
(Emotion Tagged Dataset)
https://www.kaggle.com/datasets/parulpandey/emotion-dataset?select=validation.csv

(Spotify Songs)
https://www.kaggle.com/datasets/imuhammad/audio-features-and-lyrics-of-spotify-songs?resource=download



In [3]:

# using nfx to remove the stopwords from the emotion taggde data
directory = 'Emotion_training.csv'
emotion_df = pd.read_csv(directory)

# breaking it into composite emotions as a test
sadness = emotion_df[emotion_df["label"] == 0] 

s = list(map(str, sadness["text"]))
s_new = [' '.join(s)]
emotion_df['label'].value_counts()

emotion_df["clean_text"] = emotion_df['text'].apply(nfx.remove_userhandles)
emotion_df["clean_text"] = emotion_df['text'].apply(nfx.remove_stopwords)

emotion_df

Xfeatures = emotion_df["text"]
ylabels = emotion_df["label"]


# scikit learn train test split of 70-30
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.3,random_state=42)

# scikit learn pipeline with logistic regression and count vectorizer
# logistic regression is good for multiple tags for data instead of a binary pos neg classification
pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression())])

# fit the model 
pipe_lr.fit(x_train,y_train)

#function to predict the emotion of a list of lyrics 
def predict_text(model, text_arr):
    emotions_enum = ["sadness", "joy", "love", "anger" , "fear", "Surprise"]
    
    
    res = list(model.predict(text_arr))
    i = 0
    for r in res:  
        print(f"Example {i + 1}: {emotions_enum[r]}")
        i+=1
    print()
    return 

    

ex1 = "I feel so (unsure) As I take ...your hand and lead you to the dance floor As the music dies, something in your eyes Calls to mind a silver screen And all its sad good-byes Im never gonna dance again Guilty feet have got no rhythm Though it's easy to pretend I know you're not a fool I should've known better than to cheat a friend And waste the chance that I'd been given So I'm never gonna dance again The way I danced with you, oh"
ex2 = "I hate you"
ex3 = "Are you hearing little noises Maybe I’m losing my mind Trying to convince you that the voice appearing time to time are mine because " 
ex4 = "Day after Day I couldn’t stand to see you anymore I tried to silence you by burying my heart under the floor Should I have left behind that broken door Instead of peeking through the cracks and waiting "
ex5 = "I can't prove it haunts her I guess on come on. Oh switch it haunts her I guess on Come on Come. Baby you can you can you can you can you can. Olive Garden at my baby I guess on Come on Come. Tell 'em too bad your body on come on come on."
predict_text(pipe_lr, [ex1, ex2, ex3, ex4, ex5])


Example 1: sadness
Example 2: joy
Example 3: joy
Example 4: sadness
Example 5: sadness



/Users/tejashariharan/opt/anaconda3/envs/CSCI3832/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
# this returns the actual string of the emotion
def predict_text_clean(model, text_single):
    emotions_enum = ["sadness", "joy", "love", "anger" , "fear", "Surprise"]
    res = list(model.predict(text_single))
    return emotions_enum[res[0]]


directory = 'spotify_songs.csv'
df = pd.read_csv(directory)
df_small = pd.DataFrame({
    'Artist': df['track_artist'],
    'Lyrics': df['lyrics']
})

# appending it to the song dataset
a = []
for i in df_small["Lyrics"][0:100]:
    a.append(predict_text_clean(pipe_lr, [i]))


df_smaller = df_small.head(100)

df_smaller.insert(2, "Emotion", a, True)
df_smaller

,Artist,Lyrics,Emotion
0,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,anger
1,Steady Rollin,"The trees, are singing in the wind The sky blu...",love
2,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",fear
3,CeeLo Green,I really can't stay Baby it's cold outside I'v...,anger
4,KARD,Get up out of my business You don't keep me fr...,sadness
...,...,...,...
95,Ariana Grande,"Mmm, yeah, yuh I thought you into my life (Who...",joy
96,R3HAB,I jump the line And you're always waiting for ...,sadness
97,Lianne La Havas,I am neon Cold neon Not a mystery All I've eve...,sadness
98,Rex Orange County,How could I ignore you? Trust me I adore you W...,sadness
